# Agenda

1. Modules + packages
2. Basic objects
    - `class`
    - What happens when we define a class? What is a class, anyway?
    - Attributes -- the core of objects in Python
    - Attribute lookup via ICPO
    - Magic methods
    - Inheritance
    - Methods vs. functions
    - Properties
    - Descriptors

# Modules

Modules do two distinct things in Python:

1. They provide us with a library system, so that we can write code once and then use it many times. We can reuse our own code, and others can use our code, and we can use other people's code.
2. They provide us with namespaces.

# We use `import` to load a module

When we use `import`, we're really doing two or three things:

- Python loads the file associated with the name we're importing (this happens once)
- Python creates a module object based on the file, and stores it
- We get a global varible referring to that module object

Note that when we say `import modname`, we are not putting `modname` in quotes! We are not giving a path to a file, or anything else that could be interpeted as a string.We're giving the name of the variable we want to define.

In [1]:
import random

In [2]:
# Where does Python look for the "random" module? In "random.py". 
# Python looks, one by oine, through every directory in sys.path, and the first place it finds random.py, that file is loaded.

import sys
sys.path

['/Users/reuven/Courses/Current/Cisco-2024-07July-advanced',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python312.zip',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/lib-dynload',
 '',
 '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/site-packages']

In [3]:
random

<module 'random' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/random.py'>

If you set the environment variable PYTHONPATH, that can contain the names of one or more directories where Python should also look for modules. Those will go after the standard library, but before site-packages.

# Caching of modules

The first time we say `import random`, Python finds the file, loads it, stores it in memory, and defines the variable.

The second and other times we say `import random`, Python sees that we already loaded it, and doesn't load it again. Rather, it just defines the variable to refer to the module object.

Where does Python cache this? In a dictionary, `sys.modules`. The keys to this dict are strings, the names of the modules, and the values are the module objects.

In [4]:
sys.modules

{'sys': <module 'sys' (built-in)>,
 'builtins': <module 'builtins' (built-in)>,
 '_frozen_importlib': <module '_frozen_importlib' (frozen)>,
 '_imp': <module '_imp' (built-in)>,
 '_thread': <module '_thread' (built-in)>,
 '_warnings': <module '_warnings' (built-in)>,
 '_weakref': <module '_weakref' (built-in)>,
 '_io': <module '_io' (built-in)>,
 'marshal': <module 'marshal' (built-in)>,
 'posix': <module 'posix' (built-in)>,
 '_frozen_importlib_external': <module '_frozen_importlib_external' (frozen)>,
 'time': <module 'time' (built-in)>,
 'zipimport': <module 'zipimport' (frozen)>,
 '_codecs': <module '_codecs' (built-in)>,
 'codecs': <module 'codecs' (frozen)>,
 'encodings.aliases': <module 'encodings.aliases' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/encodings/aliases.py'>,
 'encodings': <module 'encodings' from '/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/encodings/__init__.py'>,
 'encodings.utf_8': <module 'encodings.utf_8' from '/Users/reuven/.pyenv/versi

In [5]:
dir(random)

['BPF',
 'LOG4',
 'NV_MAGICCONST',
 'RECIP_BPF',
 'Random',
 'SG_MAGICCONST',
 'SystemRandom',
 'TWOPI',
 '_ONE',
 '_Sequence',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_accumulate',
 '_acos',
 '_bisect',
 '_ceil',
 '_cos',
 '_e',
 '_exp',
 '_fabs',
 '_floor',
 '_index',
 '_inst',
 '_isfinite',
 '_lgamma',
 '_log',
 '_log2',
 '_os',
 '_pi',
 '_random',
 '_repeat',
 '_sha512',
 '_sin',
 '_sqrt',
 '_test',
 '_test_generator',
 '_urandom',
 '_warn',
 'betavariate',
 'binomialvariate',
 'choice',
 'choices',
 'expovariate',
 'gammavariate',
 'gauss',
 'getrandbits',
 'getstate',
 'lognormvariate',
 'normalvariate',
 'paretovariate',
 'randbytes',
 'randint',
 'random',
 'randrange',
 'sample',
 'seed',
 'setstate',
 'shuffle',
 'triangular',
 'uniform',
 'vonmisesvariate',
 'weibullvariate']

In [6]:
# if I say `import random` a second time, Python sees that it has that module ('random' in sys.modules),
# and so it just has to say

random = sys.modules['random']   # this defines the variable for the module object

# Variations on `import`

- `import MODNAME` -- loads the module (if needed), assigns the variable `MODNAME` to the module object. `MODNAME.py` is the filename that Python will look for in `sys.path`.
- `from MODNAME import NAME` -- loads the module (if needed), it assigns the variable `NAME` to `sys.modules[MODNAME].name`. It doesn't define `MODNAME` as a variable. We do this for convenience, so that we can write shorter variable/method names in our programs.
- `import MODNAME as ALIAS` -- this does the same as `import MODNAME`, but the variable that is assigned is `ALIAS`, not `MODNAME`.
- `from MODNAME import NAME as ALIAS` -- this does the same as `from .. import`, but only assigns one global variable, and it's `ALIAS`, referring to the individual attribute `NAME` on the module object.
- `from MODNAME import *` -- this imports *all* of the names in `MODNAME` as global variables into the current module. **NEVER EVER EVER EVER EVER EVER EVER EVER DO THIS!!**

In [7]:
import time

In [8]:
time

<module 'time' (built-in)>

In [9]:
from random import randint

In [ ]:
# from numpy import *
# from pandas import *
# from random import *
# from argparse import *

# __version__

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "pandas" that refers to sys.modules['pandas']

import pandas   

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "pd" that refers to sys.modules['pandas']

import pandas as pd

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it defines a variable "Series" that refers to sys.modules['pandas'].Series

from pandas import Series

In [ ]:
# Python will
# (1) look for pandas.py in each of the directories in sys.path. 
# (2) Assuming it finds such a file, it loads it into memory, executes it, top to bottom, and then stores the result as a module object
#     in sys.modules.
# (3) Then it runs a for loop over every name in sys.modules['pandas'], and defines a variable for each one

from pandas import *

for one_name in dir(sys.modules['pandas']):
    globals()[one_name] = getattr(sys.modules['pandas'], one_name)    

In [10]:
from pandas import elephant

ImportError: cannot import name 'elephant' from 'pandas' (/Users/reuven/.pyenv/versions/3.12.1/lib/python3.12/site-packages/pandas/__init__.py)

In [11]:
import mymod

In [12]:
mymod

<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [14]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [15]:
mymod.__file__

'/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'

In [16]:
mymod.__name__

'mymod'

In [17]:
import mymod

In [18]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [20]:
from importlib import reload

In [21]:
reload(mymod)  # this deletes sys.modules['mymod'], so that we can import it again

<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [22]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'hello',
 'x',
 'y']

In [23]:
mymod.x   # x is an attribute on the mymod object

10

In [24]:
mymod.y

[10, 20, 30]

In [25]:
mymod.hello('world')

'Hello, world!'

In [26]:
reload(mymod)

Hello
Goodbye


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [27]:
reload(mymod)

Hello from mymod
Goodbye from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

In [28]:
dir(mymod)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'hello',
 'x',
 'y']

In [29]:
reload(mymod)

Hello from mymod
Goodbye from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

# `__name__` and `'__main__'`

The `__name__` variable always exists in Python, and it always contains a string indicating the current namespace. If you are in a module that was loaded with `import`, then the value of `__name__` will be the string of the file that was loaded. For example, when we said `import mymod`, the value of `__name__` inside of the module was the string `'mymod'`.

But the first module/program to be loaded into Python when it starts up has a special name, the string `'__main__'`. Don't confuse this with the `main` function in C or other programming languages! This simply means that we're in the first program to be loaded, and that no one imported us.

- `__name__` is a variable
- `'__main__'` is a string value

So what? We can then test the value of `__name__` in our module. If it's `'__main__'`, then we know the module is being run as a program, and not be imported as a module.

We can then have a module that also offers interactive features. The module's defintions will always be loaded, but only the stuff after the `if __name__ == '__main__'` will be executed when we're interactive.

In [30]:
reload(mymod)

Hello from mymod


<module 'mymod' from '/Users/reuven/Courses/Current/Cisco-2024-07July-advanced/mymod.py'>

# What's the use case for `if __name__ == '__main__'`?

- Run automated tests
- Start up a server
- Have the module demo its capabilities
- Offer interactive services based on the module's functionality

Above that line, you define variables, classes, and functions. Those will always be defined, whether it's run interactively or via `import`. But the stuff below that line will only be run interactively, when you execute the module as a program.

# Exercise: Menu function

Create a module, `menu.py`, which defines a single function, `menu`. This function takes any number of string arguments. Those are the options that the user can choose from. The idea is that someone writing a larger program wants to force the user to choose from among several predefined options. They can call `menu('a', 'b', 'c')`, and the user will be asked to enter a, b, or c.

The user's choice will be returned to the caller.

You should be able to write code like this:

```python
import menu

user_choice = menu.menu('a', 'b', 'c')   # user will be asked to choose from a, b, or c; their choice will be returned + assigned
print(f'User chose {user_choice}')
```

Also make it possible to execute the module from the command line. In such a case, it'll call `menu` with x, y, and z, and ask the user to choose from them. The user's choice will be displayed.

In [33]:
reload(menu)

user_choice = menu.menu('a', 'b', 'c')   # user will be asked to choose from a, b, or c; their choice will be returned + assigned
print(f'User chose {user_choice}')


Choose (a/b/c):  x


x is not a valid option; try again


Choose (a/b/c):  a


User chose a


# Packages

There are two types of "packages" in Python:

- Regular packages, which are folders/directories containing modules.
- Distribution packages, which allow PyPI and `pip` to download and install code on our computer.

A distribution package is a zipfile/wheelfile containing a directory with some meta-data, including dependencies. That directory contains a subdirectory which is the "regular" package, containing one or more modules.

# Next up: Objects!

Return at :35

# What is an object?

We like to say that everything in Python is an object. But how can we define an object? All objects have three qualities:

1. ID, which we can retrieve via the `id` builtin function.
2. A type, which we can retrieve via the `type` builtin.
3. Attributes, which we can get a list of via `dir`.

The attributes can contain data or functions. 

In [34]:
# we can use "type" to get the type of an object

type(10)

int

In [35]:
type('abcd')

str

In [37]:
type([10, 20, 30])

list

In [39]:
# when we get a type/class back from calling type(), we don't get a string 
# describing the type of object. Rather, we get an actual object, str, int, or list.

# if they're objects, then they must have types!
type(int) 

type

In [40]:
type(str)

type

In [41]:
type(list)

type

Every object in Python has a type, which we can retrieve with `type`. The type of every class in Python is `type`. 

Attributes are a private dict on an object. Every object has attributes.

We can get a list of attributes with the `dir` function.  We can retrieve the value of an attribute on an object with the dot syntax:

    a.b

Here, we're saying that we want the value of the `b` attribute on the object `a`.    

We could also write

    getattr(a, 'b')   # builtin function

What about setting an attribute? We can do that, as well -- becauase it's a dict that belongs to an object, creating and updating an attribute's value are both done via assignment:

    a.x = 100    # this either sets or updates the value of the "x" attribute to be 100

Or say:

    setattr(a, 'x', 100)  # builtin function

# Variables vs. attributes

Variables exist in either local or global scopes, and we can tell they are variables because there are no `.` characters before their names.

By contrast, attributes belong to an object. They don't belong to a variable, and don't have a scope. Almost always, an attribute has a `.` before its name, showing us the object to which it belongs.

So if we say

    a.b

Then `a` is a variable (local or global). We retrieve the object to which the variable refers, and then ask the object, "Do you have the attribute `b`?" If so, then we get the value back. If not, then we get an `AttributeError` exception.    



# Defining a class

If we want to define a new class, we do that with the `class` keyword. A class is an object that can create new objects, or instances. If we have a class `Foo`, then any object it creates is "an instance of `Foo`."

Can we define an empty class? Yes!

In [42]:
class MyClass:
    pass   # this is needed because we need an indented block after the :, but don't want to say anything

In [43]:
type(MyClass)

type

In [44]:
# how do I create a new instance of MyClass?
# we always invoke the class to get a new instance back

m1 = MyClass()
m2 = MyClass()

In [45]:
type(m1)  # what kind of data does m1 contain?

__main__.MyClass

In [46]:
type(m2)

__main__.MyClass

In [47]:
vars(m1)   # what attributes have been set specifically on m1?

{}

In [48]:
dir(m1)   # what attributes are available via m1, which includes inheritance and such?

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [49]:
# we can always ask an object to tell us who it inherits from
# by asking for the .__bases__ attribute. This gives us a tuple,
# typically with only one element.

MyClass.__bases__

(object,)

We've seen that `m1` and `m2` are legit instances of `MyClass`, but don't have any attributes.

Attributes in Python are used where other languages talk about instance variables ("fields"), class variables, instance methods, and class methods. All of these fall under the "attribute" umbrella. 

If we want the equivalent of an instance variable in Python, we assign it to the object.

In [50]:
m1.x = 100
m1.y = [10, 20, 30]

m2.a = 'hello'
m2.b = {'a':10, 'b':20}

In [51]:
# thanks to the fact that I can assign any attribute to any value in Python,
# I have now added two attributes (x and y) to m1, and two other attributes (a and b)
# to m2

vars(m1)

{'x': 100, 'y': [10, 20, 30]}

In [52]:
vars(m2)

{'a': 'hello', 'b': {'a': 10, 'b': 20}}

This is a terrible idea! The whole point of an object system is that every instance of a particular type has identical attributes, and thus identical (or similar) behavior. 

What we really want is a way to say: When we create a new object of `MyClass`, we want to immediately and automatically assign some attributes so that we won't have this kind of mess.

This method is known as an "initializer." (No, it's not the constructor.)

In [53]:
class MyClass:
    def __init__(self, x, y):   # this is actually MyClass.__init__
        self.x = x
        self.y = y

m1 = MyClass(10, 20)  # MyClass.__new__(10, 20) --> MyClass.__init__(new_instance, 10, 20)  --> o.x = 10, o.y = 20
m2 = MyClass('abcd', 'efgh')

In [54]:
vars(m1)

{'x': 10, 'y': 20}

In [55]:
vars(m2)

{'x': 'abcd', 'y': 'efgh'}

# What happens when we create a new instance?

1. We invoke the class, calling it. Here, we invoke `MyClass()`, but we pass arguments to it. Those arguments are grabbed by `*args` and `**kwargs`, so we can pass more or less whatever we want.
2. The method that is actually invoked by calling the class is the constructor method, which creates a new object. Its name is `__new__`. It is very, *very* rare for you to actually write a `__new__` method. The signature of `__new__` takes `*args` and `**kwargs`. It is here that the new object is created; memory is allocated it, and the type is assigned to it. You can think of this as a new, "naked" object.
3. Before `__new__` returns the new object to the caller, it first invokes `__init__`, the initializer. Just as the job of `__new__` is to create a new object, the job of `__init__` is to add attributes to the new object. By the time `__init__` is called, the object exists. `__new__` calls `__init__`, passing `*args` and `**kwargs` as arguments (unrolled) to the method.
4. `__init__`, as you see, has a parameter named `self`. That is how we pass the instance itself to the method. The word `self` is a convention, not a requirement, but everyone uses it. `self` is a parameter in `__init__`, and thus is a local variable to that method. The value passed (assigned) to it is the newly created instance from `__new__`.
5. `__init__` takes `self` (the new instance) and its other arguments and assigns as necessary/appropriate the arguments to attributes on the object. Because they're attributes, their names all comes after `.`. And because they're attributes on the instance, they are all `self.` and then a name.
6. `__init__` doesn't even have to `return`. The mere fact that it assigned attributes to the object is enough. When it's done, `__new__` can then return the new object (aka `self`) to its caller. Now it's not only a new object, but it has one or more attributes assigned.

In [56]:
# variables are the keys in the globals() and locals() dicts
# attributes are the keys in an object's own dict, which we can see with dir(the_object)

def hello():  # hello is a variable
    return 'Hello!'

import random
random.hello = hello   # now I've assigned the "hello" function to the attribute "hello" in the "random" module

In [58]:
class MyClass:
    def __init__(self, x, y): 
        self.x = x
        self.y = y

m1 = MyClass(10, 20)
print(m1.x)    # here, I turn to the object "m" and ask: Do you have an attribute "x"? Yes, value is 10
print(m1.y)       # here, I turn to the object "m" and ask: Do you have an attribute "y"? Yes, value is 20

m2 = MyClass('a', 'b')
print(m2.x)
print(m2.y)

10
20


Shouldn't we have "getter" and "setter" methods?

In Python, almost always no. 

If you want to retrieve an attribute from an object, just retrieve it. If you want to set/update a value on an attribute, just set it.

When we get to properties and descriptors, we'll see that we can limit things a bit.

If we don't need getters and setters, then do we need methods? Sure, to perform calculations. Or produce reports. Or do more complex things than just setting and retrieving.

# Exercise: Ice cream

1. Define a `Scoop` class. When you create an instance of `Scoop`, you will pass one string, the flavor that you want to give it. That will be assigned to the `flavor` attribute on the instance.
2. Create three instances of `Scoop`, each with its own flavor.
3. Define a `Bowl` class. Each instance of `Bowl` will have a single attribute, `scoops`, a list into which we can put one or more `Scoop` objects.
4. The `Bowl` class doesn't take any arguments when you create an instance, but it does have a `add_scoops` method, which takes any number of scoops, and adds them to the `scoops` attribute.
5. The `Bowl` class also needs a `flavors` method, which returns a list of strings, the flavors from the scoops in the `scoops` attribute.

Example:

    s1 = Scoop('chocolate')
    s2 = Scoop('vanilla')
    s3 = Scoop('coffee')

    b = Bowl()
    b.add_scoops(s1, s2, s3)
    print(b.flavors())  # ['chocolate', 'vanilla', 'coffee']

In [64]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')

class Bowl:
    def __init__(self):
        self.scoops = []
    def add_scoops(self, *new_scoops):
        self.scoops += new_scoops
    def flavors(self):
        # output = []
        # for one_scoop in self.scoops:
        #     output.append(one_scoop.flavor)
        # return output

        # I have a list of scoops
        # I want a list of flavors
        # the expression I can use to get one from the other is .flavor
        # let's use a list comprehension!
        return [one_scoop.flavor
                for one_scoop in self.scoops]

b = Bowl()
b.add_scoops(s1, s2, s3)
print(b.flavors())

['chocolate', 'vanilla', 'coffee']


In [65]:
# let's explore a Person class

class Person:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1
Hello, name2


In [66]:
# my boss says that we need to add a new feature: Keeping track of 
# the population of Person objects we've created

population = 0    # global variable to track population

class Person:
    def __init__(self, name):
        self.name = name
        population += 1
    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, {population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {population=}')

print(p1.greet())
print(p2.greet())

Before, population=0


UnboundLocalError: cannot access local variable 'population' where it is not associated with a value

In [67]:
# Let's fix this with attributes!
# everything in Python is an object
# (that includes classes)
# every object in Python has attributes
# and we can assign/update any attribute on any object

class Person:
    def __init__(self, name):
        self.name = name
        Person.population += 1
    def greet(self):
        return f'Hello, {self.name}'

Person.population = 0    # now it's an attribute on the Person class

print(f'Before, {Person.population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')

print(p1.greet())
print(p2.greet())

Before, Person.population=0
After, Person.population=2
Hello, name1
Hello, name2


In [68]:
# here's another class

print('A')
class MyClass:
    print('B')
    def __init__(self, x):
        print('C')
        self.x = x
    print('D')
print('E')

m1 = MyClass(10)
m2 = MyClass(20)

# when I run this code, what letters will be printed, and in what order?

A
B
D
E
C
C


When we define our `MyClass` class, `__init__` is not assigned to a variable. Rather, it's assigned to an attribute, to `MyClass.__init__`. 

The only explanation that works is: Inside of a `class` definition, assignemnts are not made to variables. Rather, they're made to attributes on the class. When we have `def __init__` inside of `class MyClass`, we're really defining `MyClass.__init__`.

If we were to have any variable assignments inside of the `class` block, we would really be assigning to attributes on the class, not to variables.

Modules work the same way: We assign to variables inside of the module file, but when we `import` the module, we actually have attributes on the module object assigned.

You can think of a class as a module without an external file.

In [69]:
# With this knowledge, we can now define population *inside* of the Person class!

class Person:
    population = 0    # this is *not* a variable! Not global, not local. This is Person.population, an attribute on the class

    def __init__(self, name):
        self.name = name
        Person.population += 1
    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, {Person.population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')

print(p1.greet())
print(p2.greet())

Before, Person.population=0
After, Person.population=2
Hello, name1
Hello, name2


# Why/where do we want class attributes?

1. Methods are all class attributes.
2. If we have a shared resource that is most appropriately put in a class.
3. Constants, or semi-constants, that make sense for the class and its instances.

In [70]:
# we know that Person, p1, and p2 are distinct objects, each with their own attributes.
# But maybe, just maybe, we can access the class attribute "population" via the instances?

class Person:
    population = 0    

    def __init__(self, name):    # methods are all class attributes
        self.name = name
        Person.population += 1
    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, {Person.population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')
print(f'After, {p1.population=}')
print(f'After, {p2.population=}')

print(p1.greet())   # methods are invoked via the instance (normally)
print(p2.greet())

Before, Person.population=0
After, Person.population=2
After, p1.population=2
After, p2.population=2
Hello, name1
Hello, name2


# Attribute search path (ICPO)

When we ask an object for an attribute, it normally looks on itself, on the instance.

If the instance has the attribute we request, then we get the value back.

If not, though, then we turn to the class of that instance, and check if the attribute is there.

In this particular case:

- We ask `p1` if it has an attribute `population`. The answer: No.
- We ask `type(p1)`, meaning `Person`, if it has an attribute `population`. The answer: Yes, the value is 2.
- We ask `p2` if it has an attribute `population`. The answer: No.
- We ask `type(p2)`, meaning `Person`, if it has an attribute `population`. The answer: Yes, the value is 2.

Things have to work this way, so that methods can be called on the instance.

- When we call `p1.greet()`, Python turns to `p1` and asks: Do you have an attribute `greet`? The answer: No.
- We ask `type(p1)`, aka `Person`, if it has an attribute `greet`. The answer: Yes, here's the method, and I'll invoke it on the instance.

In [71]:
p1.greet()

'Hello, name1'

In [72]:
# it's exactly the same to say
Person.greet(p1)

'Hello, name1'

In [73]:
p1.greet = lambda: 'hahahaha hijacked your greeting'

In [74]:
p2.greet()

'Hello, name2'

In [75]:
p1.greet()

'hahahaha hijacked your greeting'

In [77]:
# Why are we using Person.population inside of __init__?
# Couldn't we just assign to self.population instead?

# class attributes: You can definitely retrieve them via self,
# but you should never assign to them via self, because you'll
# create an instance attribute that shadows/blocks access to
# the class attribute from the instance.

class Person:
    population = 0    

    def __init__(self, name):  
        self.name = name
        self.population += 1     # don't do this! 
    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, {Person.population=}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, {Person.population=}')
print(f'After, {p1.population=}')
print(f'After, {p2.population=}')

print(p1.greet()) 
print(p2.greet())

Before, Person.population=0
After, Person.population=0
After, p1.population=1
After, p2.population=1
Hello, name1
Hello, name2


# Next up

1. Inheritance
2. (more) Magic methods
3. Properties
4. Descriptors

Resume at 13:30 Paris Time

In [78]:
class Person:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [80]:
# The boss says: We need a new class, Employee, which is the same as Person
# with one exception: Every employee needs an ID number.

class Person:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

class Employee:
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
    def greet(self):
        return f'Hello, {self.name}!'

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())
print(e2.greet())

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


# What is inheritance?

If one class inherits from another, it means that the "child class" is basically the same as the "parent class," except in specific places.

The whole idea of inheritance is to avoid reinventing the wheel, copying and pasting, etc. We want to DRY up (don't repeat yourself) our code.

Here, our `Employee` class is basically the same as our `Person` class. As such, we can have `Employee` inherit from `Person`, only tweaking/changing where needed.

There are basically two types of relationships in object-oriented programming:
- `has-a` -- composition, where we have one object inside of another. The string (flavor) inside of a `Scoop` object. The string (`name`) insde of a `Person` object. Composition is by far the more common and popular relationship.
- `is-a` -- inheritance, where one class can be said to be a more specific version of another class. Here, we could say that `Employee` is-a `Person`. 

In [81]:
# let's use inheritance!

class Person:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

class Employee(Person):   # Employee is-a Person, inherits from Person
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number
    def greet(self):
        return f'Hello, {self.name}!'

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())
print(e2.greet())

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


In [82]:
# we can check: What class does each of Person and Employee inherit from?

Employee.__bases__  # this is a tuple containing our parent class(es)

(__main__.Person,)

In [83]:
Person.__bases__  

(object,)

In [84]:
object.__bases__

()

# Attribute lookup (again)

Before, we said that attributes are searched for in two places:

- `I`, the instance that we name in our expression (e.g., `p1.name` or `p1.population`)
- `C`, the class of that instance (e.g., `Person.population` or `Person.__init__`)
- `P`, the parent of the class (e.g., `Person` in the case of `Employee`) -- there could be many layers of parents
- `O`, the `object` class, at the top of the hierarchy.

In [90]:
# now that Employee is-a Person, let's take advantage of that and modify our class definitions

class Person:
    def __init__(self, name):
        self.name = name
    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

class Employee(Person):   # Employee is-a Person, inherits from Person
    def __init__(self, name, id_number):
        # Person.__init__(self, name)   
        super().__init__(name)   # this calls the __init__ method on our parent, passing it name (and self, behind the scenes)
        self.id_number = id_number

e1 = Employee('emp1', 1) # Does e have __init__? No. Does Employee have __init__? Yes.
e2 = Employee('emp2', 2)

print(e1.greet())  # does e1 have greet? No. Does e1's class (Employee) have greet? No! Does Person have greet? Yes!
print(e2.greet())

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


# Method inheritance paradigms

1. If the child class wants the same behavior as the parent class, just don't have a method in the child class.
2. If the child class wants the parent's behavior and *ALSO* to add their own behavior, then we first run the parent method (with `super`), then do what we want/need in the child method.
3. If the child class wants only its own behavior, then just define a method in the child class and don't call `super`.

# Exercise: Ice cream (part 2)

1. Let's limit the number of scoops that someone can add to their bowl to 3. (You only need to worry about adding scoops via `add_scoops`, not assigning to the list.) You'll want a class attribute indicating the maximum (`MAX_SCOOPS`), and then to use that maximum when adding scoops. If the user tries to add too many, then just ignore the extra scoops.
2. Now that we've upset all of our young customers with the small bowl size, we'll offer a new product, a `BigBowl`, which is just the same as `Bowl`, but can handle up to 5 scoops, instead.

In [97]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')

class Bowl:
    MAX_SCOOPS = 3
    def __init__(self):
        self.scoops = []
    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:
            if len(self.scoops) >= self.MAX_SCOOPS:
                break
            self.scoops.append(one_scoop)
    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

b = Bowl()
b.add_scoops(s1, s2, s3)
b.add_scoops(s4, s5)
print(b.flavors())

class BigBowl(Bowl):
    MAX_SCOOPS = 5

bb = BigBowl()
bb.add_scoops(s1, s2, s3)
bb.add_scoops(s4, s5)
bb.add_scoops(s6)
print(bb.flavors())

['chocolate', 'vanilla', 'coffee']
['chocolate', 'vanilla', 'coffee', 'flavor 4', 'flavor 5']


In [98]:
class Person:
    def __init__(self, name):
        self.name = name

p1 = Person('name1')
p2 = Person('name2')

# what happens when I do this:
print(p1)
print(p2)

In [100]:
hex(id(p1))

'0x12650f860'

# String representations

When we invoke `print` on an object in Python, the first thing it does is call `str` on the argument. That's how we're able to print every type! Every object knows how to turn itself into a string. This is handled by the `object.__str__` method, via inheritance.

Meaning:

- We invoke `print(p1)`
- Behind the scenes, this means `print(str(p1))`
- That is translated into `print(p1.__str__())`, a method call
    - Does `p1` have an attribute `__str__`? No.
    - Does `Person` have an attribute `__str__`? No.
    - Does `object` have an attribute `__str__`? Yes, and that's the method which is invoked.
- However, if we define a method called `__str__` on `Person`, then the search will be terminated sooner, and we'll get a custom string back.

This is the first of many "magic methods" we'll be discussing, also known as "dunder methods." The idea is that you can change the behavior of every operator in Python, as well as many other aspects of the language, by defining these methods in your classes. There about 100 such methods; you will not ever need to implement even one third of them.

In [101]:
class Person:
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return f'Person with name {self.name}'

p1 = Person('name1')
p2 = Person('name2')

# what happens when I do this:
print(p1)
print(p2)

Person with name name1
Person with name name2


In [102]:
# now in Jupyter, I can print p1
print(p1)

Person with name name1


In [105]:
# I can also just ask for p1's printed representation by writing its name in Jupyter
p1

There are *two* ways to turn an object into a string:

- When we use `str` or `print`, we are invoking `__str__`, to get a string that's appropriate for the user.
- When we're in Jupyter and are looking at the printed representation, that's actually returned by a separate method, `__repr__`, which you can see by invoking `repr` on something.

The idea is that `__str__` is meant for "normal" users, whereas `__repr__` is meant for people using debuggers, etc.

In theory, `__repr__` is supposed to return a string that's a legal Python expression. In practice, I tell people to only implement `__repr__`, since if it's defined, it'll take over for itself and also `__str__`.

If and when you ever want to distinguish between what programmers see and the general users see, you can always define a new `__str__` method.

In [106]:
class Person:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f'Person with name {self.name}'

p1 = Person('name1')
p2 = Person('name2')

# what happens when I do this:
print(p1)
print(p2)

Person with name name1
Person with name name2


In [107]:
p1

Person with name name1

In [108]:
p2

Person with name name2

In [109]:
# one more magic method: __len__
# this is what the len() builtin function looks for on an object!
# if you have a countable object, you can implement __len__. 

class Person:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f'Person with name {self.name}'
    def __len__(self):
        return len(self.name)

p1 = Person('name1')
p2 = Person('name2')

# what happens when I do this:
print(p1)
print(p2)

Person with name name1
Person with name name2


In [110]:
len(p1)

5

In [111]:
len(p2)

5

# Exercise: Magic methods

1. Add `__repr__` to both `Scoop` and `Bowl`. In the case of `Scoop`, it should return a string indicating the flavor. In the case of `Bowl`, it should return a multi-string string numbering each of the scoops and indicating its flavor.
2. Add `__len__` to `Bowl`, so that you can find out how many scoops are in a bowl.

In [118]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')

class Bowl:
    MAX_SCOOPS = 3
    def __init__(self):
        self.scoops = []
    def add_scoops(self, *new_scoops):
        for one_scoop in new_scoops:
            if len(self.scoops) >= self.MAX_SCOOPS:
                break
            self.scoops.append(one_scoop)
    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]
    def __repr__(self):
        output = [f'Scoops in {type(self).__name__}:']
        for index, one_scoop in enumerate(self.scoops, 1):
            output.append(f'\t{index}: {one_scoop}')

        return '\n'.join(output)
    def __len__(self):
        return len(self.scoops)

b = Bowl()
b.add_scoops(s1, s2, s3)
b.add_scoops(s4, s5)
print(b.flavors())

class BigBowl(Bowl):
    MAX_SCOOPS = 5

bb = BigBowl()
bb.add_scoops(s1, s2, s3)
bb.add_scoops(s4, s5)
bb.add_scoops(s6)
print(bb.flavors())

print('-' * 60)
print(s1)
print(s2)
print(b)
print(bb)

['chocolate', 'vanilla', 'coffee']
['chocolate', 'vanilla', 'coffee', 'flavor 4', 'flavor 5']
------------------------------------------------------------
Scoop of chocolate
Scoop of vanilla
Scoops in Bowl:
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee
Scoops in BigBowl:
	1: Scoop of chocolate
	2: Scoop of vanilla
	3: Scoop of coffee
	4: Scoop of flavor 4
	5: Scoop of flavor 5


In [120]:
len(bb)

5

In [121]:
class MyClass:
    def __init__(self, x):
        self.x = x

m1 = MyClass('a')
m2 = MyClass('a')

m1 == m2

False

In [124]:
# in order to convince == that two things have the same value,
# we need to implement the __eq__ method.

class MyClass:
    def __init__(self, x):
        self.x = x
    def __eq__(self, other):
        if hasattr(other, 'x'):
            return self.x == other.x   # now we'll compare their x attributes
        else:
            return False

m1 = MyClass('a')
m2 = MyClass('a')

m1 == m2

True

In [125]:
m1 == 'hello'

False

In [130]:
# how is it that strings, lists, tuples, and dicts all use [] to retrieve
# from the data? The answer: [] invoke the __getitem__ method.


class MyClass:
    def __init__(self, x):
        self.x = x
    def __getitem__(self, index):
        # print(index)
        return self.x[index]

m = MyClass('abcdefg')
m[3]

'd'

In [131]:
m[2:4]

'cd'

In [ ]:
def myfunc():
    g[1] = 5    # this is translated into g.__setitem__(1, 5)

# All dunder methods

https://www.pythonmorsels.com/every-dunder-method/

# Next up

1. Chart out the Python object system
2. Properties
3. Descriptors

Resume at :15

# Properties



In [132]:
class Thermostat:
    def __init__(self, temp=20):
        self.temp = temp

t = Thermostat()
print(t.temp)
t.temp = 22
print(t.temp)

20
22


In [144]:
# change our Thermostat class, such that people cannot set the temperature < 10 or > 30.

# a property is a class attribute that works like methods
# to the outside world, it feels like we're just setting/getting data
# but internally, we're running methods with all of the functionality

class Thermostat:
    def __init__(self, temp=20):
        self._temp = temp     # _ at the start of a name means, "private"

    @property   # this turns the "temp" method into a property-getter method, invoked *WITHOUT* parentheses
    def temp(self):
        print(f'Now retrieving _temp via the getter method')
        return self._temp

    @temp.setter
    def temp(self, new_temp):
        print(f'Now setting {new_temp} as the new temperature')

        if new_temp < 10:
            raise ValueError('Too low! Lowest is 10')

        if new_temp > 30:
            raise ValueError('Too high! Highest is 30')

        self._temp = new_temp

In [145]:
# our property is a class attribute
# which we access via the instance
# and thus behaves (thanks to descriptor magic) like a method, even though it looks like attribute

t = Thermostat()
print(t.temp)

Now retrieving _temp via the getter method
20


In [146]:
t.temp = 22

Now setting 22 as the new temperature


In [147]:
print(t.temp)

Now retrieving _temp via the getter method
22


In [148]:
t.temp = 50

Now setting 50 as the new temperature


ValueError: Too high! Highest is 30

In [149]:
t.temp = -40

Now setting -40 as the new temperature


ValueError: Too low! Lowest is 10

In [155]:
help(property)

Help on class property in module builtins:

class property(object)
 |  property(fget=None, fset=None, fdel=None, doc=None)
 |
 |  Property attribute.
 |
 |    fget
 |      function to be used for getting an attribute value
 |    fset
 |      function to be used for setting an attribute value
 |    fdel
 |      function to be used for del'ing an attribute
 |    doc
 |      docstring
 |
 |  Typical use is to define a managed attribute x:
 |
 |  class C(object):
 |      def getx(self): return self._x
 |      def setx(self, value): self._x = value
 |      def delx(self): del self._x
 |      x = property(getx, setx, delx, "I'm the 'x' property.")
 |
 |  Decorators make defining new properties or modifying existing ones easy:
 |
 |  class C(object):
 |      @property
 |      def x(self):
 |          "I am the 'x' property."
 |          return self._x
 |      @x.setter
 |      def x(self, value):
 |          self._x = value
 |      @x.deleter
 |      def x(self):
 |          del self._x
 |
 |

In [ ]:
# there is a __del__  magic method that you can set, which is invoked when the object
# is about to be freed. But you don't know when that'll happen! 

with open('/etc/passwd') as f:
    # here, we invoke f.__enter__()  
    pass
    # here, we invoke f.__exit__(), which in the case of a file flushes + closes the file object

# Properties are special (easy) cases of *descriptors*

If I have three classes, and they all need the same property, then it's hard/annoying to have the same code in all three classes. I would prefer to define one class and then use instances of that class in each of my overall classes.

This is possible, thanks to the *descriptor protocol* in Python. It says:

- If you have a class attribute ("the descriptor")
- And you access it via an instance
- And the descriptor class defines `__get__`
- Then that `__get__` method is invoked, and its result is returned instead of the descriptor object.
- Or if you assign to it, and the descriptor class defines `__set__`,
- Then that `__set__` method is invoked instead.

In other words, we have a way to hijack retrieving a class attribute or setting it, so long as we do so via the instance.

So we have three objects here:

- Descriptor class, where we implement `__get_